 ASS-3 Fine-tune GPT or GPT-2 for creative story generation.

 Uses transformers and datasets

In [ ]:
pip install transformers datasets

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

print('Libraries imported.')

Libraries imported.


Creates a small dataset of short stories.

In [ ]:
# Create a small dummy dataset for creative story generation
data = {
    "text": [
        "Once upon a time, in a land far away, there was a mysterious forest that whispered secrets.",
        "The brave knight ventured into the enchanted woods, seeking the lost treasure of the ancient king.",
        "In a small village, a young dreamer wrote magical tales that came to life under the starlight."
    ]
}

dataset = Dataset.from_dict(data)

# Load the GPT-2 tokenizer and set a padding token
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=64)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set the format to PyTorch tensors
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

print('Data prepared and tokenized.')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Data prepared and tokenized.


Converts text into GPT-2-compatible format.

In [ ]:
# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Prepare data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set training arguments (using small values for a quick demo)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10,
    save_total_limit=2,
    logging_steps=5,
    prediction_loss_only=True,
    remove_unused_columns=False
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# Fine-tune the model
print('Starting fine-tuning...')
trainer.train()
print('Fine-tuning complete.')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pallavi-22320177 (pallavi-22320177-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Fine-tuning complete.


Trains the model on the dataset.

After fine-tuning, GPT-2 learns from the given stories and generates similar text. Instead of random output, it produces more story-like sentences.

In [ ]:
# Generate text using the fine-tuned model
prompt = "In a world of magic, "
inputs = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated Story:\n")
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generated Story:

In a world of magic,  the power of the world had to offer was not limited to the power to create the magic. It was also the only power that could be used to make the universe.
The world was a place where the people lived. The people who lived in the lives of people. They were the ones who had the knowledge and the ability to use the powers. And the beings who were those who could use magic to bring about the things that were needed. Those who
